## Record Matching Algorithm with an example (two datasets over one decade)

* We will display one possible approach to match records using the existing rltk algorithm and Gale Shapley matching algorithm in this file 

In [1]:
import rltk
import pandas as pd

In [2]:

ds1 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1850 from 1850-1860 cleaned.csv')
ds2 = pd.read_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1860 from 1850-1860 cleaned.csv')

In [3]:
ds1["matching_id"] = None
ds2["matching_id"] = None


#### In order for rltk algorithm to properly work, we want to make sure each of the important columns we choose must be in string type.

In [4]:
ds1['histid'] = ds1['histid'].astype(str)
ds1['namefrst'] = ds1['namefrst'].astype(str)
ds1['namelast'] = ds1['namelast'].astype(str)
ds1['bpl'] = ds1['bpl'].astype(str)

ds2['histid'] = ds2['histid'].astype(str)
ds2['namefrst'] = ds2['namefrst'].astype(str)
ds2['namelast'] = ds2['namelast'].astype(str)
ds2['bpl'] = ds2['bpl'].astype(str)

In [5]:
def create_dataframe(dataset1, dataset2, histid_weight = 0.4, firstname_weight = 0.2, lastname_weight = 0.2, blp_weight = 0.2):

    df = {index1: [] for index1, row1 in dataset1.iterrows()}
    

    for index1, row1 in dataset1.iterrows():
        score_list = []
        
        
        histid1 = row1['histid']
        firstname1 = row1['namefrst']
        lastname1 = row1['namelast']
        birthplace1 = row1['bpl']

        for index2, row2 in dataset2.iterrows():

            histid2 = row2['histid']
            firstname2 = row2['namefrst']
            lastname2 = row2['namelast']
            birthplace2 = row2['bpl']

            histid_score =  rltk.levenshtein_similarity(histid1, histid2)*histid_weight
            firstname_score = rltk.levenshtein_similarity(firstname1, firstname2)*firstname_weight
            lastname_score = rltk.levenshtein_similarity(lastname1, lastname2)*lastname_weight

            if birthplace1 == birthplace2:
                birthplace_score = 1*blp_weight
            else:
                birthplace_score = 0*blp_weight

            total_score = histid_score + firstname_score + lastname_score + birthplace_score
            
            score_list.append(total_score)

        df[index1].extend(score_list)

    return pd.DataFrame(df)


#### Using the rltk algorithm to help us create a matrix showing a linkage score between any pair of records in two datasets. Higher the score, the more likely the two records become a match. For example, the record 0 in dataset1 and the record 1 from dataset2 has a score of 0.168889.

In [6]:
score_df = create_dataframe(ds1, ds2)
create_dataframe(ds1, ds2)


,0,1,2,3,4,5,6,7,8,9,...,387,388,389,390,391,392,393,394,395,396
0,0.125000,0.100000,0.133333,0.111111,0.100000,0.158333,0.127778,0.136111,0.158333,0.113889,...,0.138889,0.100000,0.161111,0.147222,0.122222,0.111111,0.325000,0.077778,0.122222,0.155556
1,0.111111,0.138889,0.077778,0.113889,0.106667,0.188889,0.202222,0.179365,0.100000,0.127778,...,0.135238,0.117460,0.134921,0.084127,0.128571,0.120238,0.306349,0.111111,0.088889,0.111111
2,0.066667,0.077778,0.088889,0.169444,0.088889,0.066667,0.111111,0.066667,0.088889,0.080556,...,0.077778,0.077778,0.055556,0.066667,0.155556,0.127778,0.277778,0.117778,0.066667,0.077778
3,0.111111,0.305556,0.084127,0.109127,0.161905,0.100000,0.184127,0.180952,0.100000,0.130556,...,0.152381,0.117460,0.106349,0.106349,0.173016,0.156349,0.128571,0.066667,0.122222,0.077778
4,0.122222,0.100000,0.151111,0.202778,0.066667,0.100000,0.088889,0.112698,0.122222,0.125000,...,0.157460,0.155556,0.095238,0.066667,0.133333,0.066667,0.135556,0.384444,0.066667,0.088889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,0.088889,0.122222,0.111111,0.136111,0.136111,0.113889,0.088889,0.205556,0.100000,0.100000,...,0.122222,0.144444,0.111111,0.088889,0.111111,0.088889,0.111111,0.183333,0.088889,0.122222
680,0.102778,0.102778,0.161111,0.080556,0.150000,0.088889,0.091667,0.116667,0.169444,0.227778,...,0.091667,0.111111,0.102778,0.125000,0.088889,0.080556,0.113889,0.138889,0.116667,0.077778
681,0.077778,0.106349,0.128571,0.141667,0.152381,0.134921,0.095238,0.112698,0.184127,0.188889,...,0.095238,0.122222,0.084127,0.106349,0.122222,0.102778,0.077778,0.084127,0.112698,0.122222
682,0.084127,0.113889,0.066667,0.113889,0.117460,0.146032,0.150794,0.066667,0.100000,0.141667,...,0.077778,0.361905,0.284127,0.284127,0.339683,0.345238,0.084127,0.117460,0.128571,0.088889


#### We then include Gale Shapley algorithm below, and this algorithm returns a list of matching pairs based on the best scores. Each pair consists of one record from one dataset and one record from another dataset. Some records might not find a matching, so its matching record is None in that pair. We will also attach the associated score given each successful match in the list. 

In [7]:
def gale_shapley(score_df):
    # Get the list of row and column names
    proposing_set = score_df.index.tolist()
    receiving_set = score_df.columns.tolist()
    
    # Initialize dictionaries to store engagements
    engagements = {}
    reverse_engagements = {}

    # Initialize all elements as free
    free_proposers = proposing_set[:]
    for receiver in receiving_set:
        engagements[receiver] = None

    while free_proposers:
        proposer = free_proposers.pop(0) # get the list element at the a specific position
        # and reduce list without that element in place (this is just one row in dataset A)
        proposals = score_df.loc[proposer].sort_values(ascending=False).index.tolist()
        # we find the best match of this row from dataset A in dataset B and we will finalize it in the following...

        for receiver in proposals:
            if receiver not in reverse_engagements.values():
                engagements[receiver] = proposer
                reverse_engagements[proposer] = receiver
                break

            else:
                current_proposer = engagements[receiver] 
                if score_df.loc[proposer, receiver] > score_df.loc[current_proposer, receiver]:
                    engagements[receiver] = proposer
                    reverse_engagements[proposer] = receiver
                    free_proposers.append(current_proposer)
                    break
    

    matches = [(proposer, receiver) for receiver, proposer in engagements.items()]
    matches_with_scores = []
    for pair in matches:
        if pair[0] is not None and pair[1] is not None:
            matches_with_scores.append((pair[0], pair[1], score_df.iloc[pair[0], pair[1]]))
        else:
            matches_with_scores.append(pair)
    
    return matches_with_scores

In [8]:
matches = gale_shapley(score_df)
matches

[(389, 0, 0.32222222222222224),
 (6, 1, 0.6555555555555556),
 (602, 2, 0.32380952380952377),
 (276, 3, 0.4083333333333333),
 (47, 4, 0.6555555555555556),
 (234, 5, 0.41333333333333333),
 (30, 6, 0.6777777777777778),
 (236, 7, 0.4714285714285715),
 (484, 8, 0.3222222222222222),
 (368, 9, 0.41111111111111115),
 (18, 10, 0.35),
 (372, 11, 0.32222222222222224),
 (278, 12, 0.43968253968253973),
 (284, 13, 0.4650793650793651),
 (241, 14, 0.39555555555555555),
 (16, 15, 0.5416666666666667),
 (9, 16, 0.5603174603174603),
 (17, 17, 0.6666666666666667),
 (20, 18, 0.711111111111111),
 (228, 19, 0.39444444444444443),
 (21, 20, 0.6555555555555556),
 (22, 21, 0.43333333333333335),
 (539, 22, 0.43666666666666665),
 (57, 23, 0.48888888888888893),
 (149, 24, 0.6666666666666667),
 (199, 25, 0.3920634920634921),
 (72, 26, 0.6888888888888889),
 (69, 27, 0.7),
 (154, 28, 0.346031746031746),
 (509, 29, 0.4838888888888889),
 (88, 30, 0.4666666666666667),
 (343, 31, 0.3904761904761905),
 (85, 32, 0.5066666666

#### This function below will help us get rid of pairs whose records fail to find a matching. 

In [9]:
def clean_pairs(matches_with_scores):
    return [pair for pair in matches_with_scores if len(pair) == 3]

In [10]:
clean_matchings = clean_pairs(matches)
clean_matchings

[(389, 0, 0.32222222222222224),
 (6, 1, 0.6555555555555556),
 (602, 2, 0.32380952380952377),
 (276, 3, 0.4083333333333333),
 (47, 4, 0.6555555555555556),
 (234, 5, 0.41333333333333333),
 (30, 6, 0.6777777777777778),
 (236, 7, 0.4714285714285715),
 (484, 8, 0.3222222222222222),
 (368, 9, 0.41111111111111115),
 (18, 10, 0.35),
 (372, 11, 0.32222222222222224),
 (278, 12, 0.43968253968253973),
 (284, 13, 0.4650793650793651),
 (241, 14, 0.39555555555555555),
 (16, 15, 0.5416666666666667),
 (9, 16, 0.5603174603174603),
 (17, 17, 0.6666666666666667),
 (20, 18, 0.711111111111111),
 (228, 19, 0.39444444444444443),
 (21, 20, 0.6555555555555556),
 (22, 21, 0.43333333333333335),
 (539, 22, 0.43666666666666665),
 (57, 23, 0.48888888888888893),
 (149, 24, 0.6666666666666667),
 (199, 25, 0.3920634920634921),
 (72, 26, 0.6888888888888889),
 (69, 27, 0.7),
 (154, 28, 0.346031746031746),
 (509, 29, 0.4838888888888889),
 (88, 30, 0.4666666666666667),
 (343, 31, 0.3904761904761905),
 (85, 32, 0.5066666666

#### Lastly, we establish thresholds to classify each successful match as a Yes(meaning it is almost certain that the two records are a match), Maybe(the two records could be a match), or No (the two records are most likely not a match) match.

In [11]:
def label_matches(matching_pairs_with_scores, first_threshold = 0.1, second_threshold = 0.5):
    # Sort the list of tuples based on the score from highest to lowest
    sorted_matches = sorted(matching_pairs_with_scores, key=lambda x: x[2], reverse=True)
    
    # Determine the number of elements in each label category
    total_pairs = len(sorted_matches)
    yes_threshold = int(first_threshold * total_pairs)
    maybe_threshold = int(second_threshold * total_pairs)
    
    # Initialize lists to store the labeled matches (Yes, Maybe, or No match)
    yes_matches = sorted_matches[:yes_threshold]
    maybe_matches = sorted_matches[yes_threshold:maybe_threshold]
    no_matches = sorted_matches[maybe_threshold:]
    
    return yes_matches, maybe_matches, no_matches

In [12]:
label_matches(clean_matchings) # so, in this toy example, we have 0 Yes match, 1 Maybe match, and 2 No matches. 

([(631, 350, 0.7222222222222223),
  (20, 18, 0.711111111111111),
  (175, 76, 0.711111111111111),
  (69, 27, 0.7),
  (51, 36, 0.7),
  (204, 154, 0.7),
  (666, 367, 0.7),
  (668, 369, 0.7),
  (108, 268, 0.6936507936507937),
  (72, 26, 0.6888888888888889),
  (325, 190, 0.6888888888888889),
  (30, 6, 0.6777777777777778),
  (139, 64, 0.6777777777777778),
  (181, 82, 0.6777777777777778),
  (638, 93, 0.6777777777777778),
  (252, 112, 0.6777777777777778),
  (207, 160, 0.6777777777777778),
  (208, 161, 0.6777777777777778),
  (213, 162, 0.6777777777777778),
  (217, 178, 0.6777777777777778),
  (662, 200, 0.6777777777777778),
  (422, 217, 0.6777777777777778),
  (424, 219, 0.6777777777777778),
  (299, 246, 0.6777777777777778),
  (300, 247, 0.6777777777777778),
  (581, 316, 0.6777777777777778),
  (610, 336, 0.6777777777777778),
  (611, 337, 0.6777777777777778),
  (17, 17, 0.6666666666666667),
  (149, 24, 0.6666666666666667),
  (128, 52, 0.6666666666666667),
  (143, 62, 0.6666666666666667),
  (7, 75,

#### Note: the threshold to classify and label matches as well as the criteria to compute the linkage score before can be changed flexibly based on need.

In [13]:
ds1['matching_certainty'] = None
ds2['matching_certainty'] = None

In [14]:
new_id = 0
matching_Level = ["Yes", "Maybe", "No"]
for certainty_level, level_tuples in zip(matching_Level, label_matches(clean_matchings)):
    for tuple in level_tuples:
        ds1.loc[tuple[0], "matching_id"] = new_id
        ds2.loc[tuple[1], "matching_id"] = new_id
        new_id += 1

        ds1.loc[tuple[0], "matching_certainty"] = certainty_level
        ds2.loc[tuple[1], "matching_certainty"] = certainty_level

    

##### Now we can export our datasets with matching ids 

In [15]:
ds1.to_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1850_matching.csv', index = False)

In [16]:
ds2.to_csv('C:/Columbia_University/Research/History_African_American_Migration_Pattern/US_MLP_datasets/1860_matching.csv', index = False)